In [1]:
import numpy as np
import tensorflow as tf
#import cv2
#import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
from os import listdir
from os.path import isfile, join
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")    #for tensorboard
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [2]:
def _parse_function(example_proto):
    features = {
                "image_y": tf.FixedLenFeature((), tf.string ),
                "image_m": tf.FixedLenFeature((), tf.string )
                #"image_x": tf.FixedLenFeature((), tf.string )
                }

    parsed_features = tf.parse_single_example(example_proto, features)
    
    image_y = tf.decode_raw(parsed_features["image_y"],  tf.float64)
    image_m = tf.decode_raw(parsed_features["image_m"],  tf.float64)
    
    image_y = tf.reshape(image_y, [512,512,1])
    image_m = tf.reshape(image_m, [512,512,1])
    #image_x = tf.decode_raw(parsed_features["image_x"],  tf.float64)
    tf.summary.image("64_Y",image_y,3)
    tf.summary.image("64_M",image_m,3)
    
    image_y = tf.cast(image_y,dtype=tf.float32)
    image_m = tf.cast(image_m,dtype=tf.float32)
    #image_x = tf.cast(image_x,dtype=tf.float32)
    tf.summary.image("32_Y",image_y,3)
    tf.summary.image("32_M",image_m,3)
    
    #image_y = tf.reshape(image_y, [512,512,1])
    #image_m = tf.reshape(image_m, [512,512,1])
    #image_x = tf.reshape(image_x, [512,512,1])
    
    return image_y,image_m

In [3]:
def batch_norm(inputs, is_training, decay=.99, epsilon=0.00000001):
    with tf.name_scope("batch_norm") as scope:


        scale = tf.get_variable("scale_BN", (inputs.get_shape()[1:4]), initializer=tf.ones_initializer())
        beta = tf.get_variable("beta_BN", (inputs.get_shape()[1:4]), initializer=tf.zeros_initializer())
        pop_mean = tf.get_variable("pop_mean", (inputs.get_shape()[1:4]), initializer=tf.zeros_initializer(), trainable=False)
        pop_var = tf.get_variable("pop_var", (inputs.get_shape()[1:4]), initializer=tf.ones_initializer(), trainable=False)

        mean = tf.cond(tf.cast(is_training,tf.bool), lambda: tf.nn.moments(inputs,[0])[0], lambda: tf.multiply(tf.ones(inputs.get_shape()[1:4]), pop_mean))
        var = tf.cond(tf.cast(is_training,tf.bool), lambda: tf.nn.moments(inputs,[0])[1], lambda: tf.multiply(tf.ones(inputs.get_shape()[-1]), pop_var))
        train_mean = tf.cond(tf.cast(is_training,tf.bool), lambda:tf.assign(pop_mean, pop_mean*decay+mean*(1-decay)),lambda:tf.zeros(1))
        train_var = tf.cond(tf.cast(is_training,tf.bool),lambda:tf.assign(pop_var, pop_var*decay+var*(1-decay)),lambda:tf.zeros(1))

        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(inputs, mean, var, beta, scale, epsilon)

In [4]:



def partial_conv(pixel, mask,is_training, kernel_size, filter_numbers, stride, batch_n, nonlinearity, trans):

    with tf.name_scope("part_conv") as scope:
        kernel_h = kernel_size[0]
        kernel_w = kernel_size[1]
        if trans==True:
            kernel_d = filter_numbers
            kernel_o = pixel.get_shape().as_list()[3]
        elif trans==False:
            kernel_d = pixel.get_shape().as_list()[3]
            kernel_o = filter_numbers
        elif trans=="same_pad":
            #kernel_d = pixel.get_shape().as_list()[3]
            #kernel_o = filter_numbers
            kernel_d = filter_numbers
            kernel_o = pixel.get_shape().as_list()[3]
            
            
        W = tf.get_variable('Weights', (kernel_h, kernel_w, kernel_d, kernel_o),
                            initializer=tf.contrib.layers.variance_scaling_initializer())
        
        
        W1 = tf.ones((kernel_h, kernel_w, kernel_d, kernel_o), name='Weights_mask')

        Z1 = tf.multiply(pixel, mask, name="element_op")

        if trans==True:
            #need to fix for variable last batch size. The last mini_batch will be of different size most of the time
            out_shape_list = pixel.get_shape().as_list()
            out_shape_list[1] = pixel.get_shape().as_list()[1] + 2
            out_shape_list[2] = pixel.get_shape().as_list()[2] + 2
            out_shape_list[3] = filter_numbers
            out_shape = tf.constant(out_shape_list)
            #out_shape = tf.TensorShape(out_shape_list)
            #out_shape = tf.cast(out_shape,tf.int32)
            prime_conv = tf.nn.conv2d_transpose(Z1, W,out_shape, strides=stride, padding="VALID", name="prime_conv")
            sec_conv = tf.nn.conv2d_transpose(mask, W1,output_shape=tf.TensorShape(out_shape_list), strides=stride, padding="VALID", name="sec_conv")
        elif trans==False:
            prime_conv = tf.nn.conv2d(Z1, W, strides=stride, padding="VALID", name="prime_conv")
            sec_conv = tf.nn.conv2d(mask, W1, strides=stride, padding="VALID", name="sec_conv")
        elif trans=="same_pad":
            #prime_conv = tf.nn.conv2d(Z1, W, strides=stride, padding="SAME", name="prime_conv")
            #sec_conv = tf.nn.conv2d(mask, W1, strides=stride, padding="SAME", name="sec_conv")
            out_shape_list = pixel.get_shape().as_list()
            out_shape_list[1] = pixel.get_shape().as_list()[1] 
            out_shape_list[2] = pixel.get_shape().as_list()[2] 
            out_shape_list[3] = filter_numbers
            out_shape = tf.constant(out_shape_list)
            prime_conv = tf.nn.conv2d_transpose(Z1, W,out_shape, strides=stride, padding="SAME", name="prime_conv")
            sec_conv = tf.nn.conv2d_transpose(mask, W1,output_shape=tf.TensorShape(out_shape_list), strides=stride, padding="SAME", name="sec_conv")


        inver_sum = tf.divide(tf.constant(1.0), sec_conv)
        clean_sum = tf.where(tf.is_inf(inver_sum), tf.zeros_like(inver_sum), inver_sum)

        weighted_pixel = tf.multiply(prime_conv, clean_sum, name="multi_inver_sum")
        up_mask = tf.where(tf.not_equal(sec_conv, tf.constant(0.0)),tf.ones_like(sec_conv),sec_conv)

        #normalized_out = tf.cond(tf.cast(batch_n,tf.bool), lambda:batch_norm(weighted_pixel, is_training), lambda:weighted_pixel)
        B = tf.get_variable('Biases',(1,weighted_pixel.get_shape()[1],weighted_pixel.get_shape()[2],weighted_pixel.get_shape()[3]),
                            initializer=tf.constant_initializer(.01))
        
        normalized_out = tf.add(weighted_pixel,B)
        #normalized_out = weighted_pixel
        
        if nonlinearity=="relu":
            up_pixel = tf.nn.relu(normalized_out, name="relu")
        elif nonlinearity=="leaky_relu":
            up_pixel = tf.nn.leaky_relu(normalized_out, name="leaky_relu")
        elif nonlinearity=="none":
            up_pixel = normalized_out
            #up_pixel = tf.sigmoid(normalized_out)
            #up_pixel = tf.nn.relu(normalized_out, name="relu")
            
        tf.summary.histogram("weights", W)    
        tf.summary.histogram("biases", B)   
        tf.summary.histogram("activations", up_pixel)   
        
        return up_pixel, up_mask
    


def place_holders(mini_size,height, width, channels):
    #X = tf.placeholder(tf.float32, shape=(mini_size, height, width, channels))
    Y = tf.placeholder(tf.float32, shape=(mini_size, height, width, channels))
    M = tf.placeholder(tf.float32, shape=(mini_size, height, width, channels))
    return M ,Y


def near_up_sampling(pixel, mask, output_size):
    with tf.name_scope("nearest_up") as scope:
        up_pixel = tf.image.resize_nearest_neighbor(pixel, size=output_size, name="nearest_pixel_up")
        up_mask = tf.image.resize_nearest_neighbor(pixel, size=output_size, name="nearest_mask_up")
        return up_pixel, up_mask

def concat(near_pixel, pconv_pixel, near_mask, pconv_mask):
    with tf.name_scope("concatenation") as scope:
        up_pixel = tf.concat([pconv_pixel, near_pixel], axis=3)
        up_mask = tf.concat([pconv_mask,near_mask], axis=3)
        return up_pixel, up_mask

def decoding_layer(pixel_in,mask_in,is_training, output_size_in, pconv_pixel1, pconv_mask1, filter_numbers1):
    with tf.name_scope("decoding") as scope:
        near_pixel1,near_mask1 = near_up_sampling(pixel_in,mask_in,output_size_in)
        concat_pixel,concat_mask = concat(near_pixel1, pconv_pixel1, near_mask1, pconv_mask1)
        pixel_out,mask_out = partial_conv(concat_pixel,concat_mask,is_training,[3,3],filter_numbers1,[1,1,1,1],
                                        True,"leaky_relu",trans=True)
        return pixel_out,mask_out


In [5]:
def forward_prop(is_training, pixel, mask):

    with tf.variable_scope("PConv1") as scope:
        p_out1,m_out1 = partial_conv(pixel,mask,is_training,kernel_size=[7,7],filter_numbers=64,stride=[1,2,2,1],
                                    batch_n=False,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv2") as scope:
        p_out2,m_out2 = partial_conv(p_out1,m_out1,is_training,kernel_size=[5,5],filter_numbers=128,stride=[1,2,2,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv3") as scope:
        p_out3,m_out3 = partial_conv(p_out2,m_out2,is_training,kernel_size=[5,5],filter_numbers=256,stride=[1,2,2,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv4") as scope:
        p_out4,m_out4 = partial_conv(p_out3,p_out3,is_training,kernel_size=[3,3],filter_numbers=512,stride=[1,2,2,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv5") as scope:
        p_out5,m_out5 = partial_conv(p_out4,m_out4,is_training,kernel_size=[3,3],filter_numbers=512,stride=[1,2,2,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv6") as scope:
        p_out6,m_out6 = partial_conv(p_out5,m_out5,is_training,kernel_size=[3,3],filter_numbers=512,stride=[1,2,2,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv7") as scope:
        p_out7,m_out7 = partial_conv(p_out6,m_out6,is_training,kernel_size=[3,3],filter_numbers=512,stride=[1,1,1,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("PConv8") as scope:
        p_out8,m_out8 = partial_conv(p_out7,m_out7,is_training,kernel_size=[3,3],filter_numbers=512,stride=[1,1,1,1],
                                    batch_n=True,nonlinearity="relu",trans=False)
    with tf.variable_scope("decoding9") as scope:
        p_out9,m_out9 = decoding_layer(p_out8,m_out8,is_training,(p_out7.get_shape().as_list()[1],p_out7.get_shape().as_list()[2]),
                                        p_out7,m_out7,filter_numbers1=512)

    with tf.variable_scope("decoding10") as scope:
        p_out10,m_out10 = decoding_layer(p_out9,m_out9,is_training,(p_out6.get_shape().as_list()[1],p_out6.get_shape().as_list()[2]),
                                        p_out6,m_out6,filter_numbers1=512)

    with tf.variable_scope("decoding11") as scope:
        p_out11,m_out11 = decoding_layer(p_out10,m_out10,is_training,(p_out5.get_shape().as_list()[1],p_out5.get_shape().as_list()[2]),
                                        p_out5,m_out5,filter_numbers1=512)

    with tf.variable_scope("decoding12") as scope:
        p_out12,m_out12 = decoding_layer(p_out11,m_out11,is_training,(p_out4.get_shape().as_list()[1],p_out4.get_shape().as_list()[2]),
                                        p_out4,m_out4,filter_numbers1=512)

    with tf.variable_scope("decoding13") as scope:
        p_out13,m_out13 = decoding_layer(p_out12,m_out12,is_training,(p_out3.get_shape().as_list()[1],p_out3.get_shape().as_list()[2]),
                                        p_out3,m_out3,filter_numbers1=256)

    with tf.variable_scope("decoding14") as scope:
        p_out14,m_out14 = decoding_layer(p_out13,m_out13,is_training,(p_out2.get_shape().as_list()[1],p_out2.get_shape().as_list()[2]),
                                        p_out2,m_out2,filter_numbers1=128)

    with tf.variable_scope("decoding15") as scope:
        p_out15,m_out15 = decoding_layer(p_out14,m_out14,is_training,(p_out1.get_shape().as_list()[1],p_out1.get_shape().as_list()[2]),
                                        p_out1,m_out1,filter_numbers1=64)

    #with tf.variable_scope("decoding16") as scope:
    #    p_out16,m_out16 = decoding_layer(p_out15,m_out15,is_training,(pixel.get_shape().as_list()[1],pixel.get_shape().as_list()[2]),
    #                                    pixel,mask,filter_numbers1=1)

    
    
    
    with tf.variable_scope("decoding16") as scope:
        near_pixel1,near_mask1 = near_up_sampling(p_out15,m_out15,(pixel.get_shape().as_list()[1],pixel.get_shape().as_list()[2]))
        concat_pixel,concat_mask = concat(near_pixel1, pixel, near_mask1, mask)
        pixel_out,mask_out = partial_conv(concat_pixel,concat_mask,is_training,[3,3],filter_numbers=1,stride=[1,1,1,1],
                                        batch_n=False,nonlinearity="none",trans="same_pad")
        return pixel_out,mask_out



def compute_cost(pixel_gt,mask_gt,pixel_pre,hole_pera):
    loss_valid = tf.losses.absolute_difference(tf.multiply(pixel_gt,mask_gt),tf.multiply(pixel_pre,mask_gt), weights=1.0,
                                                reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS)
    loss_hole = tf.losses.absolute_difference(tf.multiply(pixel_gt,(1-mask_gt)),tf.multiply(pixel_pre,(1-mask_gt)), weights=1.0,
                                                reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS)

    total_loss = (loss_valid + tf.multiply(hole_pera,loss_hole))/7.0
    
    tf.summary.scalar('loss',total_loss)
   
    return total_loss


In [6]:
def model(learning_rate,num_epochs,mini_size,break_t,break_v,pt_out):
    #ops.reset_default_graph()
#     m = x_train.shape[0]
#     h = x_train.shape[1]
#     w = x_train.shape[2]
#     c = x_train.shape[3]
    m = 9882
    #m = 8
    h = 512
    w = 512
    c = 1
    
    m_val_size = 1098
    #m_val_size = 2
    costs = []

    M,Y = place_holders(mini_size,h, w, c)
    
    tf.summary.image("input_Y",Y,3)
    tf.summary.image("input_M",M,3)
    
    is_training = tf.placeholder(tf.bool,name="training")
    
    pixel_out, mask_out = forward_prop(is_training,pixel=Y, mask=M)

    cost = compute_cost(pixel_gt=Y, mask_gt=M, pixel_pre=pixel_out, hole_pera=6.0)

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    init = tf.global_variables_initializer()
    
    #data_x = tf.placeholder(x_train.dtype, x_train.shape)
    #data_m = tf.placeholder(m_train.dtype, m_train.shape)
    #data_y = tf.placeholder(y_train.dtype, y_train.shape)
    
    
    
    filenames = "/media/antor/Files/ML/Papers/train_new.tfrecords"
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(_parse_function)
    #dataset = tf.data.Dataset.from_tensor_slices((data_x, data_m, data_y))
    
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.shuffle(20)
    dataset = dataset.batch(mini_size)
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()
    num_mini = int(m/mini_size)          #must keep this fully divided and num_mini output as int pretty sure it doesn't need
                                    #to be an int
    
    merge_sum = tf.summary.merge_all()
    #l1_summary = tf.summary.scalar('L1', cost)            #for tensorboard
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())   #for tensorboard
    
    saver = tf.train.Saver()
    
    #gpu_options = tf.GPUOptions(allow_growth=True)
    #session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
    sess = tf.Session()
    sess.run(init)
    #sess.run(iterator.initializer, feed_dict={data_x:x_train, data_m:m_train, data_y:y_train})
    sess.run(iterator.initializer)
    mini_cost = 0.0
    counter = 1
    
    while True:
        try:
            pix_gt, mask_in = sess.run(next_element)
            pix_gt = tf.reshape(pix_gt,[mini_size,512,512,1])
            mask_in = tf.reshape(mask_in,[mini_size,512,512,1])
            #pix_in = tf.reshape(pix_in,[mini_size,512,512,1])
            
            pix_gt = pix_gt.eval(session=sess)
            mask_in = mask_in.eval(session=sess)
            #pix_in = pix_in.eval(session=sess)
            #print( pix_gt)
            #print( mask_in)
            #print( pix_in)
            #pixel_out, mask_out = forward_prop(True,pixel=label_in, mask=mask_in)

            #cost = compute_cost(pixel_gt=pix_in, mask_gt=mask_in, pixel_pre=pixel_out, hole_pera=6.0)

            #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
            
            _ , temp_cost = sess.run([optimizer,cost],feed_dict={M:mask_in, Y:pix_gt, is_training:True})
            #_ , temp_cost = sess.run([optimizer,cost])
            
            #mini_cost += temp_cost/num_mini
            mini_cost += temp_cost/pt_out
            
            if counter%5 == 0:
                s = sess.run(merge_sum, feed_dict={M:mask_in, Y:pix_gt, is_training:True})
                file_writer.add_summary(s,counter)
            
            if counter%num_mini==0:
                print("cost after epoch " + str(counter/num_mini) + ": " + str(mini_cost))
                mini_cost =0.0 
            #print("cost after epoch " + str(counter/num_mini) + ": " + str(mini_cost))
            
#             if counter%50==0:         #for tensorboard
#                 summary_str = l1_summary.eval(session=sess,feed_dict={M:mask_in, Y:pix_gt, is_training:True})
#                 #step = int(counter%num_mini) * n_batches + batch_index
#                 file_writer.add_summary(summary_str, counter)
            
            #if counter%1==0:
            #    print("mini batch cost of batch " + str(counter) + " is : " + str(temp_cost))
            if counter%pt_out==0:
                print("mini batch cost of batch " + str(counter) + " is : " + str(mini_cost))
                mini_cost =0.0 
            
            if counter==break_t:
                break
            
            counter = counter + 1
        except tf.errors.OutOfRangeError:
            #counter = counter + 1
            #print("cost after epoch " + str(counter) + ": " + str(mini_cost))
            #mini_cost = 0.0
            break
    
    file_writer.close()            #for tensorboard
    #data_x_val = tf.placeholder(x_val.dtype, x_val.shape)
    #m_val = m_val.astype(float)
    #data_m_val = tf.placeholder(m_val.dtype, m_val.shape)
    #data_y_val = tf.placeholder(y_val.dtype, y_val.shape)
    
    filenames_val = "/media/antor/Files/ML/Papers/val_new.tfrecords"
    dataset_val = tf.data.TFRecordDataset(filenames_val)
    dataset_val = dataset_val.map(_parse_function)
    
    #dataset_val = tf.data.Dataset.from_tensor_slices((data_x_val,data_m_val,data_y_val))
    dataset_val = dataset_val.shuffle(20)
    dataset_val = dataset_val.batch(mini_size)
    
    iterator_val = dataset_val.make_initializable_iterator()
    next_element_val = iterator_val.get_next()
    num_mini_val = int(m_val_size/mini_size)

    counter_val = 1
    #sess.run(iterator_val.initializer, feed_dict={data_x_val:x_val,data_m_val:m_val,data_y_val:y_val})
    sess.run(iterator_val.initializer)
    mini_cost_val = 0.0
    #break_num = 40
    while True:
        try:
            label_in_val, mask_in_val = sess.run(next_element_val)
            #pix_in_val = tf.reshape(pix_in_val,[mini_size,512,512,1])
            mask_in_val = tf.reshape(mask_in_val,[mini_size,512,512,1])
            label_in_val = tf.reshape(label_in_val,[mini_size,512,512,1])
            #pix_in_val = pix_in_val.eval(session=sess)
            mask_in_val = mask_in_val.eval(session=sess)
            label_in_val = label_in_val.eval(session=sess)
            
            temp_cost_val = sess.run(cost, feed_dict={M:mask_in_val,Y:label_in_val,is_training:False})
            
            
            mini_cost_val += temp_cost_val/break_v
            
            if counter_val%num_mini_val==0:
                print("cost after epoch " + str(counter_val/num_mini_val) + ": " + str(mini_cost_val))
                mini_cost_val =0.0 
            counter_val = counter_val + 1
            
            if counter_val==break_v:
                print("cost of val_set " + ": " + str(mini_cost_val))
                break
            
            
        except tf.errors.OutOfRangeError:

            #print("validation cost after epoch " + "1" + ": " + str(mini_cost_val))

            break
    
    #save_path = saver.save(sess, "/home/antor/Downloads/model_checkpoint/my_model_final.ckpt")
    sess.close()


In [7]:
model(learning_rate=.055,num_epochs=1,mini_size=2,break_t=100,break_v=10,pt_out=10)

mini batch cost of batch 10 is : 0.27852838188409806
mini batch cost of batch 20 is : 0.11842885017395019
mini batch cost of batch 30 is : 0.08787534832954406
mini batch cost of batch 40 is : 0.07339720837771892
mini batch cost of batch 50 is : 0.07144180946052076
mini batch cost of batch 60 is : 0.05750089175999164
mini batch cost of batch 70 is : 0.06498565673828126
mini batch cost of batch 80 is : 0.05042915306985378
mini batch cost of batch 90 is : 0.059459491074085234
mini batch cost of batch 100 is : 0.06574115678668023
cost of val_set : 0.05998010262846946
